# SAE Session 6: System Testing and Performance Validation
## Industry 4.0 Smart Manufacturing Cell - OpenManipulator-X

**Duration**: 8 hours  
**Prerequisites**: Sessions 1-5 completed  
**Objectives**:
- Develop comprehensive test suite for manufacturing cell
- Implement performance metrics collection
- Conduct repeatability and accuracy analysis
- Validate system against specifications
- Generate quality reports

---

## Part 1: Unit Testing Framework (2 hours)

### Theory

**Software Testing Levels**:
1. **Unit Tests**: Individual functions/modules
2. **Integration Tests**: Component interactions
3. **System Tests**: Complete system behavior
4. **Acceptance Tests**: Meets requirements

For robotics systems, we also need:
- **Kinematic validation**: FK/IK correctness
- **Trajectory verification**: Smoothness, limits
- **Performance benchmarks**: Speed, accuracy, reliability

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from roboticstoolbox import DHRobot, RevoluteDH
import spatialmath as sm
import unittest
from datetime import datetime
import json

# Import your manufacturing cell modules
# from kinematics import forward_kinematics, inverse_kinematics
# from trajectory import plan_pick_trajectory, plan_inspection_trajectory
# from vision import detect_parts, evaluate_quality
# from control import ManufacturingCell

### Exercise 6.1: Kinematics Unit Tests

Create comprehensive tests for FK and IK functions.

In [ ]:
class TestKinematics(unittest.TestCase):
    """
    Unit tests for kinematics functions.
    """
    
    def setUp(self):
        """Initialize robot model before each test."""
        # TODO: Create robot instance
        pass
    
    def test_fk_home_position(self):
        """
        Test FK at home position [0, -45°, 45°, 0°].
        Expected end-effector position should match manual calculation.
        """
        q_home = np.array([0, np.deg2rad(-45), np.deg2rad(45), 0])
        # TODO: Implement test
        # T = forward_kinematics(q_home)
        # self.assertAlmostEqual(T[0, 3], expected_x, places=3)
        pass
    
    def test_fk_consistency_with_roboticstoolbox(self):
        """
        Verify our FK matches RoboticsToolbox results.
        """
        test_configs = [
            [0, 0, 0, 0],
            [np.pi/4, 0, 0, 0],
            [0, np.pi/4, -np.pi/4, 0],
        ]
        # TODO: For each config, compare our FK with robot.fkine(q)
        pass
    
    def test_ik_accuracy(self):
        """
        Test IK by solving for known positions.
        Strategy: FK(q) → T, then IK(T) → q', verify FK(q') ≈ T
        """
        test_configs = [
            [0, np.deg2rad(-45), np.deg2rad(45), 0],
            [np.deg2rad(30), np.deg2rad(-20), np.deg2rad(30), 0],
        ]
        # TODO: Implement FK → IK → FK verification
        pass
    
    def test_ik_multiple_solutions(self):
        """
        Verify IK returns multiple solutions when they exist.
        """
        # TODO: Test a pose with multiple IK solutions
        # Check that all solutions give same end-effector pose
        pass
    
    def test_ik_unreachable_pose(self):
        """
        Verify IK correctly handles unreachable poses.
        """
        # Target far outside workspace
        T_unreachable = sm.SE3(1.0, 0, 0) # 1 meter away (impossible)
        # TODO: Verify IK returns None or raises appropriate exception
        pass
    
    def test_joint_limits(self):
        """
        Verify joint limit checking.
        """
        q_invalid = np.array([np.deg2rad(200), 0, 0, 0])  # Exceeds ±180°
        # TODO: Verify function detects limit violation
        pass

# Run kinematics tests
# suite = unittest.TestLoader().loadTestsFromTestCase(TestKinematics)
# unittest.TextTestRunner(verbosity=2).run(suite)

### Exercise 6.2: Trajectory Tests

Validate trajectory planning functions.

In [ ]:
class TestTrajectoryPlanning(unittest.TestCase):
    """
    Unit tests for trajectory planning.
    """
    
    def test_trajectory_smoothness(self):
        """
        Verify trajectories have continuous derivatives.
        For quintic polynomials: q, qd, qdd should be continuous.
        """
        waypoints = np.array([
            [0, 0, 0, 0],
            [1, 0.5, -0.5, 0],
        ])
        # TODO: Generate trajectory and check derivative continuity
        pass
    
    def test_velocity_limits(self):
        """
        Verify trajectory respects velocity limits.
        """
        # TODO: Generate trajectory and check all velocities <= limits
        pass
    
    def test_acceleration_limits(self):
        """
        Verify trajectory respects acceleration limits.
        """
        # TODO: Check accelerations
        pass
    
    def test_waypoint_accuracy(self):
        """
        Verify trajectory passes through all waypoints.
        """
        # TODO: Check that trajectory visits each waypoint
        pass
    
    def test_inspection_trajectory_circular(self):
        """
        Verify inspection trajectory is actually circular.
        """
        center = np.array([0.15, 0, 0.1])
        radius = 0.05
        # TODO: Generate inspection trajectory
        # Check that all points are at distance 'radius' from 'center'
        pass
    
    def test_collision_free_path(self):
        """
        Verify collision-free path planning works.
        """
        # TODO: Create obstacle, plan path, verify no collisions
        pass

# Run trajectory tests

## Part 2: Performance Metrics Collection (2 hours)

### Theory

**Key Performance Indicators (KPIs)** for manufacturing cell:

1. **Cycle Time**: Total time per part
2. **Throughput**: Parts per hour
3. **Accuracy**: Position error at key points
4. **Repeatability**: Variation across multiple cycles
5. **Quality**: Percentage of correct classifications
6. **Energy**: Power consumption per cycle
7. **Reliability**: Success rate, failure modes

In [ ]:
class PerformanceMonitor:
    """
    Collect and analyze performance metrics during operation.
    """
    
    def __init__(self):
        self.data = {
            'cycle_times': [],
            'pick_times': [],
            'inspect_times': [],
            'place_times': [],
            'energies': [],
            'position_errors': [],
            'quality_decisions': [],  # (actual, predicted)
            'failures': [],
            'timestamps': [],
        }
    
    def record_cycle(self, cycle_data):
        """
        Record data from one complete cycle.
        
        Parameters:
        -----------
        cycle_data : dict
            {
                'total_time': float,
                'pick_time': float,
                'inspect_time': float,
                'place_time': float,
                'energy': float,
                'position_error': float,
                'quality_actual': bool,
                'quality_predicted': bool,
                'success': bool,
            }
        """
        self.data['cycle_times'].append(cycle_data['total_time'])
        self.data['pick_times'].append(cycle_data['pick_time'])
        self.data['inspect_times'].append(cycle_data['inspect_time'])
        self.data['place_times'].append(cycle_data['place_time'])
        self.data['energies'].append(cycle_data['energy'])
        self.data['position_errors'].append(cycle_data['position_error'])
        self.data['quality_decisions'].append((cycle_data['quality_actual'], 
                                                cycle_data['quality_predicted']))
        if not cycle_data['success']:
            self.data['failures'].append({
                'cycle': len(self.data['cycle_times']),
                'reason': cycle_data.get('failure_reason', 'Unknown')
            })
        self.data['timestamps'].append(datetime.now())
    
    def compute_statistics(self):
        """
        Compute performance statistics.
        """
        n_cycles = len(self.data['cycle_times'])
        
        if n_cycles == 0:
            return None
        
        # Time statistics
        cycle_times = np.array(self.data['cycle_times'])
        
        # Quality statistics
        quality_data = self.data['quality_decisions']
        true_positives = sum(1 for actual, pred in quality_data if actual and pred)
        true_negatives = sum(1 for actual, pred in quality_data if not actual and not pred)
        false_positives = sum(1 for actual, pred in quality_data if not actual and pred)
        false_negatives = sum(1 for actual, pred in quality_data if actual and not pred)
        
        stats = {
            'n_cycles': n_cycles,
            'cycle_time_mean': np.mean(cycle_times),
            'cycle_time_std': np.std(cycle_times),
            'cycle_time_min': np.min(cycle_times),
            'cycle_time_max': np.max(cycle_times),
            'throughput_per_hour': 3600 / np.mean(cycle_times) if np.mean(cycle_times) > 0 else 0,
            'energy_mean': np.mean(self.data['energies']),
            'energy_total': np.sum(self.data['energies']),
            'position_error_mean': np.mean(self.data['position_errors']),
            'position_error_max': np.max(self.data['position_errors']),
            'accuracy': (true_positives + true_negatives) / n_cycles,
            'precision': true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0,
            'recall': true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0,
            'false_positive_rate': false_positives / n_cycles,
            'false_negative_rate': false_negatives / n_cycles,
            'success_rate': 1 - len(self.data['failures']) / n_cycles,
            'n_failures': len(self.data['failures']),
        }
        
        return stats
    
    def generate_report(self, filename='performance_report.json'):
        """
        Generate detailed performance report.
        """
        stats = self.compute_statistics()
        
        report = {
            'test_date': datetime.now().isoformat(),
            'statistics': stats,
            'raw_data': {
                'cycle_times': self.data['cycle_times'],
                'energies': self.data['energies'],
                'position_errors': self.data['position_errors'],
            },
            'failures': self.data['failures'],
        }
        
        with open(filename, 'w') as f:
            json.dump(report, f, indent=2)
        
        return report
    
    def plot_performance(self):
        """
        Create performance visualization.
        """
        fig, axes = plt.subplots(2, 3, figsize=(15, 10))
        
        # Cycle time over runs
        axes[0, 0].plot(self.data['cycle_times'])
        axes[0, 0].axhline(np.mean(self.data['cycle_times']), color='r', linestyle='--', label='Mean')
        axes[0, 0].set_xlabel('Cycle Number')
        axes[0, 0].set_ylabel('Cycle Time (s)')
        axes[0, 0].set_title('Cycle Time per Run')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # Time breakdown (pie chart)
        time_breakdown = [
            np.mean(self.data['pick_times']),
            np.mean(self.data['inspect_times']),
            np.mean(self.data['place_times']),
        ]
        axes[0, 1].pie(time_breakdown, labels=['Pick', 'Inspect', 'Place'], autopct='%1.1f%%')
        axes[0, 1].set_title('Average Time Breakdown')
        
        # Energy consumption
        axes[0, 2].plot(self.data['energies'])
        axes[0, 2].set_xlabel('Cycle Number')
        axes[0, 2].set_ylabel('Energy (arb. units)')
        axes[0, 2].set_title('Energy per Cycle')
        axes[0, 2].grid(True)
        
        # Position errors
        axes[1, 0].plot(self.data['position_errors'])
        axes[1, 0].axhline(2.0, color='r', linestyle='--', label='Spec Limit (2mm)')
        axes[1, 0].set_xlabel('Cycle Number')
        axes[1, 0].set_ylabel('Position Error (mm)')
        axes[1, 0].set_title('Positioning Accuracy')
        axes[1, 0].legend()
        axes[1, 0].grid(True)
        
        # Cycle time histogram
        axes[1, 1].hist(self.data['cycle_times'], bins=20, edgecolor='black')
        axes[1, 1].set_xlabel('Cycle Time (s)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].set_title('Cycle Time Distribution')
        axes[1, 1].grid(True, axis='y')
        
        # Confusion matrix
        quality_data = self.data['quality_decisions']
        tp = sum(1 for actual, pred in quality_data if actual and pred)
        tn = sum(1 for actual, pred in quality_data if not actual and not pred)
        fp = sum(1 for actual, pred in quality_data if not actual and pred)
        fn = sum(1 for actual, pred in quality_data if actual and not pred)
        
        confusion = np.array([[tn, fp], [fn, tp]])
        im = axes[1, 2].imshow(confusion, cmap='Blues')
        axes[1, 2].set_xticks([0, 1])
        axes[1, 2].set_yticks([0, 1])
        axes[1, 2].set_xticklabels(['Defect', 'Good'])
        axes[1, 2].set_yticklabels(['Defect', 'Good'])
        axes[1, 2].set_xlabel('Predicted')
        axes[1, 2].set_ylabel('Actual')
        axes[1, 2].set_title('Quality Classification')
        
        # Add text annotations
        for i in range(2):
            for j in range(2):
                text = axes[1, 2].text(j, i, confusion[i, j],
                                      ha="center", va="center", color="black", fontsize=16)
        
        plt.colorbar(im, ax=axes[1, 2])
        plt.tight_layout()
        plt.savefig('performance_analysis.png', dpi=150)
        plt.show()

# Example usage
monitor = PerformanceMonitor()

### Exercise 6.3: Run Performance Tests

Execute multiple cycles and collect performance data.

In [ ]:
def run_performance_test(n_cycles=50):
    """
    Run manufacturing cell for n_cycles and collect metrics.
    
    Parameters:
    -----------
    n_cycles : int
        Number of parts to process
    
    Returns:
    --------
    monitor : PerformanceMonitor
        Monitor with collected data
    """
    monitor = PerformanceMonitor()
    
    # TODO: Initialize manufacturing cell
    # cell = ManufacturingCell(...)
    
    for i in range(n_cycles):
        # Generate random part (with random quality)
        part_quality_actual = np.random.rand() > 0.08  # 92% good rate
        
        # Simulate part detection
        part_pose = {
            'position': [0.2 + np.random.normal(0, 0.001), 
                        np.random.normal(0, 0.001),
                        -0.05],
            'orientation': np.random.uniform(-np.pi, np.pi)
        }
        
        # TODO: Execute pick-inspect-place cycle
        # result = cell.execute_cycle(part_pose)
        
        # Simulate result for now
        result = {
            'total_time': 8.0 + np.random.normal(0, 0.5),
            'pick_time': 3.0 + np.random.normal(0, 0.2),
            'inspect_time': 2.0 + np.random.normal(0, 0.1),
            'place_time': 2.5 + np.random.normal(0, 0.2),
            'energy': 50.0 + np.random.normal(0, 5),
            'position_error': abs(np.random.normal(0, 0.5)),  # mm
            'quality_actual': part_quality_actual,
            'quality_predicted': part_quality_actual if np.random.rand() > 0.02 else not part_quality_actual,  # 98% accuracy
            'success': np.random.rand() > 0.02,  # 98% success rate
        }
        
        monitor.record_cycle(result)
        
        if (i + 1) % 10 == 0:
            print(f"Completed {i+1}/{n_cycles} cycles")
    
    return monitor

# Run test
print("Running performance test...")
monitor = run_performance_test(n_cycles=50)

# Compute and display statistics
stats = monitor.compute_statistics()
print("\n=== PERFORMANCE STATISTICS ===")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"{key}: {value:.3f}")
    else:
        print(f"{key}: {value}")

# Plot results
monitor.plot_performance()

# Generate report
monitor.generate_report('test_results.json')
print("\nReport saved to test_results.json")

## Part 3: Repeatability Analysis (2 hours)

### Theory

**Repeatability** measures how consistently a robot returns to the same position.

**ISO 9283 Standard**: Repeatability specification
- Execute same motion n times (typically n=30)
- Measure position each time
- Compute: $RP = \bar{l} + 3S_l$
  - $\bar{l}$: Mean distance from centroid
  - $S_l$: Standard deviation

**Typical values**: 
- Industrial robots: ±0.05 mm
- Collaborative robots: ±0.1 mm
- Educational robots: ±0.5 mm

In [ ]:
def measure_repeatability(robot, q_target, n_repeats=30):
    """
    Measure positioning repeatability according to ISO 9283.
    
    Parameters:
    -----------
    robot : DHRobot
    q_target : ndarray
        Target joint configuration
    n_repeats : int
        Number of repetitions
    
    Returns:
    --------
    repeatability : float
        Repeatability value (RP) in mm
    positions : ndarray (n_repeats, 3)
        Measured end-effector positions
    """
    positions = []
    
    for i in range(n_repeats):
        # Simulate positioning with noise
        q_actual = q_target + np.random.normal(0, np.deg2rad(0.1), size=len(q_target))
        
        # Compute actual end-effector position
        T = robot.fkine(q_actual)
        pos = T.t
        positions.append(pos)
    
    positions = np.array(positions)
    
    # Compute centroid
    centroid = np.mean(positions, axis=0)
    
    # Distances from centroid
    distances = np.linalg.norm(positions - centroid, axis=1)
    
    # ISO 9283 repeatability
    l_bar = np.mean(distances)
    S_l = np.std(distances)
    RP = l_bar + 3 * S_l
    
    return RP * 1000, positions  # Convert to mm

# TODO: Test repeatability at multiple poses

### Exercise 6.4: Multi-Point Repeatability Test

Measure repeatability at various workspace locations.

In [ ]:
def workspace_repeatability_test(robot, test_points_configs):
    """
    Test repeatability at multiple workspace locations.
    
    Parameters:
    -----------
    robot : DHRobot
    test_points_configs : list of ndarray
        Joint configurations to test
    
    Returns:
    --------
    results : dict
        Repeatability data for each point
    """
    results = {
        'configurations': [],
        'positions': [],
        'repeatabilities': [],
        'worst_case': None,
        'best_case': None,
        'mean_repeatability': None,
    }
    
    print("Testing repeatability at multiple points...")
    
    for i, q in enumerate(test_points_configs):
        print(f"Point {i+1}/{len(test_points_configs)}...", end=' ')
        
        RP, positions = measure_repeatability(robot, q, n_repeats=30)
        
        # Get nominal position
        T_nominal = robot.fkine(q)
        pos_nominal = T_nominal.t
        
        results['configurations'].append(q)
        results['positions'].append(pos_nominal)
        results['repeatabilities'].append(RP)
        
        print(f"RP = {RP:.3f} mm")
    
    results['repeatabilities'] = np.array(results['repeatabilities'])
    results['worst_case'] = np.max(results['repeatabilities'])
    results['best_case'] = np.min(results['repeatabilities'])
    results['mean_repeatability'] = np.mean(results['repeatabilities'])
    
    return results

# Define test points for manufacturing cell
test_configs = [
    np.array([0, np.deg2rad(-45), np.deg2rad(45), 0]),  # Home
    np.array([np.deg2rad(30), np.deg2rad(-20), np.deg2rad(30), 0]),  # Pick position
    np.array([np.deg2rad(15), np.deg2rad(-30), np.deg2rad(40), 0]),  # Inspection
    np.array([np.deg2rad(45), np.deg2rad(-25), np.deg2rad(35), 0]),  # Bin A
    np.array([np.deg2rad(-45), np.deg2rad(-25), np.deg2rad(35), 0]), # Bin B
]

# TODO: Run repeatability test and analyze results

## Part 4: System Validation (2 hours)

### Validation Against Specifications

Compare actual performance vs requirements from SAE project specifications.

In [ ]:
# System specifications from SAE project
SPECIFICATIONS = {
    'cycle_time_max': 10.0,  # seconds
    'position_accuracy_max': 2.0,  # mm
    'repeatability_max': 1.0,  # mm
    'velocity_max': 200.0,  # mm/s
    'jerk_max': 50.0,  # rad/s³
    'success_rate_min': 0.98,  # 98%
    'quality_accuracy_min': 0.95,  # 95%
    'false_positive_max': 0.02,  # 2%
    'false_negative_max': 0.01,  # 1%
}

def validate_system(performance_stats, repeatability_results, specifications=SPECIFICATIONS):
    """
    Validate system performance against specifications.
    
    Returns:
    --------
    validation_report : dict
        Pass/fail for each specification
    """
    report = {
        'timestamp': datetime.now().isoformat(),
        'tests': [],
        'overall_pass': True,
    }
    
    # Test 1: Cycle Time
    test = {
        'name': 'Cycle Time',
        'spec': f"<= {specifications['cycle_time_max']} s",
        'measured': performance_stats['cycle_time_mean'],
        'pass': performance_stats['cycle_time_mean'] <= specifications['cycle_time_max'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    # Test 2: Position Accuracy
    test = {
        'name': 'Position Accuracy',
        'spec': f"<= {specifications['position_accuracy_max']} mm",
        'measured': performance_stats['position_error_max'],
        'pass': performance_stats['position_error_max'] <= specifications['position_accuracy_max'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    # Test 3: Repeatability
    test = {
        'name': 'Repeatability',
        'spec': f"<= {specifications['repeatability_max']} mm",
        'measured': repeatability_results['worst_case'],
        'pass': repeatability_results['worst_case'] <= specifications['repeatability_max'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    # Test 4: Success Rate
    test = {
        'name': 'Success Rate',
        'spec': f">= {specifications['success_rate_min']*100}%",
        'measured': performance_stats['success_rate'],
        'pass': performance_stats['success_rate'] >= specifications['success_rate_min'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    # Test 5: Quality Classification Accuracy
    test = {
        'name': 'Quality Accuracy',
        'spec': f">= {specifications['quality_accuracy_min']*100}%",
        'measured': performance_stats['accuracy'],
        'pass': performance_stats['accuracy'] >= specifications['quality_accuracy_min'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    # Test 6: False Positive Rate
    test = {
        'name': 'False Positive Rate',
        'spec': f"<= {specifications['false_positive_max']*100}%",
        'measured': performance_stats['false_positive_rate'],
        'pass': performance_stats['false_positive_rate'] <= specifications['false_positive_max'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    # Test 7: False Negative Rate
    test = {
        'name': 'False Negative Rate',
        'spec': f"<= {specifications['false_negative_max']*100}%",
        'measured': performance_stats['false_negative_rate'],
        'pass': performance_stats['false_negative_rate'] <= specifications['false_negative_max'],
    }
    report['tests'].append(test)
    if not test['pass']:
        report['overall_pass'] = False
    
    return report

def print_validation_report(report):
    """
    Print formatted validation report.
    """
    print("\n" + "="*70)
    print(" "*20 + "SYSTEM VALIDATION REPORT")
    print("="*70)
    print(f"Date: {report['timestamp']}")
    print("\n")
    
    print(f"{'Test':<30} {'Specification':<20} {'Measured':<15} {'Result':<10}")
    print("-"*70)
    
    for test in report['tests']:
        result_str = "✓ PASS" if test['pass'] else "✗ FAIL"
        measured_str = f"{test['measured']:.4f}"
        print(f"{test['name']:<30} {test['spec']:<20} {measured_str:<15} {result_str:<10}")
    
    print("-"*70)
    overall_str = "✓ SYSTEM PASSES ALL TESTS" if report['overall_pass'] else "✗ SYSTEM FAILED SOME TESTS"
    print(f"\nOVERALL: {overall_str}")
    print("="*70)

# TODO: Run validation using collected performance data

### Exercise 6.5: Complete Validation Test

Run all tests and generate comprehensive validation report.

In [ ]:
# Step 1: Run performance test
print("Step 1: Running performance test...")
# TODO: Replace with actual manufacturing cell execution
monitor = run_performance_test(n_cycles=50)
perf_stats = monitor.compute_statistics()

# Step 2: Run repeatability test
print("\nStep 2: Running repeatability test...")
# TODO: Create robot instance
from roboticstoolbox import DHRobot, RevoluteDH
robot = OpenManipulatorX()
repeatability_results = workspace_repeatability_test(robot, test_configs)

print(f"\nRepeatability Results:")
print(f"  Best case: {repeatability_results['best_case']:.3f} mm")
print(f"  Worst case: {repeatability_results['worst_case']:.3f} mm")
print(f"  Mean: {repeatability_results['mean_repeatability']:.3f} mm")

# Step 3: Validate against specifications
print("\nStep 3: Validating system...")
validation_report = validate_system(perf_stats, repeatability_results)
print_validation_report(validation_report)

# Step 4: Save all results
complete_report = {
    'performance': perf_stats,
    'repeatability': {
        'best': float(repeatability_results['best_case']),
        'worst': float(repeatability_results['worst_case']),
        'mean': float(repeatability_results['mean_repeatability']),
    },
    'validation': validation_report,
}

with open('complete_validation_report.json', 'w') as f:
    json.dump(complete_report, f, indent=2, default=str)

print("\nComplete report saved to: complete_validation_report.json")

# Step 5: Generate visualizations
monitor.plot_performance()

print("\nValidation test complete!")

## Summary and Deliverables

### What You Should Have Learned:

1. **Unit Testing**: Creating comprehensive tests for kinematics and trajectories
2. **Performance Metrics**: Collecting and analyzing KPIs
3. **Repeatability**: ISO 9283 standard measurement
4. **Validation**: Comparing performance vs specifications
5. **Quality Reporting**: Professional documentation

### Integration with SAE Project:

For your final project:
1. Implement complete test suite for all modules
2. Run 50+ cycle performance test
3. Measure repeatability at key workspace points
4. Generate validation report
5. Include test results in technical report

### For the Technical Report:

Add **Testing & Validation** section:
- Test methodology
- Performance metrics tables
- Repeatability analysis graphs
- Validation results vs specifications
- Discussion of any failures and improvements

---

**Next Session**: Final integration and presentation preparation